In [ ]:
import sys
import os

sys.path.append('F:/ai-ml/itapia/backend/evo_worker')
sys.path.append('F:/ai-ml/itapia/backend/shared')

os.environ['POSTGRES_HOST'] = 'localhost'
os.environ['REDIS_HOST'] = 'localhost'

In [ ]:
from app.dependencies import create_dependencies, get_backtest_context_manager
create_dependencies()

context_mng = get_backtest_context_manager()
context_mng.init_ready_contexts()

In [ ]:
import nest_asyncio
nest_asyncio.apply()

for _, context in context_mng.contexts.items():
    await context.load_data_into_memory()

In [ ]:
len(context_mng.get_ready_contexts())

In [ ]:
from app.backtest.action import MEDIUM_SWING_IDEAL_MAPPER, MEDIUM_SWING_PESSIMISTIC_MAPPER
from app.backtest.evaluator import MultiContextEvaluator

evaluator = MultiContextEvaluator(contexts=context_mng.get_ready_contexts()[-30:],
                                         aggregation_method='trim-mean')

evaluator.add_evaluate_scenario(MEDIUM_SWING_IDEAL_MAPPER, weight=0.3)
evaluator.add_evaluate_scenario(MEDIUM_SWING_PESSIMISTIC_MAPPER, weight=0.7)

In [ ]:
from app.algorithms.structure.operators.construct import RandomMaxDepthInitOperator
from itapia_common.schemas.entities.rules import SemanticType
init_opr = RandomMaxDepthInitOperator(purpose=SemanticType.DECISION_SIGNAL, max_depth=10)

In [ ]:
from app.algorithms.structure.operators.crossover import SubtreeCrossoverOperator
crossover_opr = SubtreeCrossoverOperator()

from app.algorithms.structure.operators.mutation import SubtreeMutationOperator
mutation_opr = SubtreeMutationOperator(max_subtree_depth=4,
                                       init_opr_template=RandomMaxDepthInitOperator)

from app.algorithms.structure.operators.selection import TournamentSelectionOperator
selection_opr = TournamentSelectionOperator(k=8)

from app.algorithms.structure.operators.replacement import NSGA2ReplacementOperator
replacement_opr = NSGA2ReplacementOperator()

In [ ]:
from app.algorithms.structure.engine.nsga2 import NSGA2EvoEngine

engine = NSGA2EvoEngine(evaluator=evaluator,
                        init_opr=init_opr,
                        crossover_opr=crossover_opr,
                        mutation_opr=mutation_opr,
                        selection_opr=selection_opr,
                        replacement_opr=replacement_opr,
                        seeding_rules=None)

In [ ]:
engine.run(pop_size=150, num_gen=10, pc=0.8, pm=0.25)

In [ ]:
engine.pareto_front[0].fitness

In [ ]:
engine.pareto_front[9].fitness

In [ ]:
lst = [list(engine.pareto_front[i].fitness) for i in range(len(engine.pareto_front))]

In [ ]:
import pandas as pd
df = pd.DataFrame(lst)

In [ ]:
df.iloc[:50]

In [ ]:
from itapia_common.rules.visualize import visualize_rule

In [ ]:
visualize_rule(engine.pareto_front[0].chromosome, 'ind0', False)

In [ ]:
visualize_rule(c1.chromosome, 'ind1', False)

In [ ]:
evaluator.evaluate(engine.pareto_front[49].chromosome)

In [ ]:
engine._is_similar(engine.pareto_front[0] ,engine.pareto_front[1])

In [ ]:
c1, c2 = crossover_opr(engine.pareto_front[0], engine.pareto_front[47])

In [ ]:
evaluator.evaluate(c1.chromosome)